In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
from torch.utils.data import Subset
from PIL import Image

In [17]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

NUM_CLASSES = 10

train_data = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)


device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}, Img: {train_data[0][0].size()}")

Files already downloaded and verified
Files already downloaded and verified
Device: cuda, Img: torch.Size([3, 224, 224])


In [18]:
from torchvision.models import resnet50
model = resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(2048, 10)  # переопределить, обязательно
model.fc.requires_grad = True

model.to(device)


c:\Users\Admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Admin\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Обучение на 1-2 эпохи
EPOCHS = 2
model.train()
for epoch in range(EPOCHS):
    running_loss = 0.0
    for images, labels in train_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader):.4f}")


Epoch 1, Loss: 1.2401
Epoch 2, Loss: 0.8401


In [20]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        
print(f'Accuracy on test data: {100 * correct / total:.2f}%')

Accuracy on test data: 73.91%


**DeconvNet**

In [33]:
import torch
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np

class DeconvNetVisualizer:
    def __init__(self, model, target_layer, target_filter):
        self.model = model.eval()
        self.target_layer = target_layer
        self.target_filter = target_filter
        self.activations = None
        self.gradients = None
        self._register_hooks()

    def _get_layer(self):
        module = self.model
        for attr in self.target_layer.split('.'):
            module = getattr(module, attr) if not attr.isdigit() else module[int(attr)]
        return module

    def _forward_hook(self, module, input, output):
        self.activations = output.detach()

    def _backward_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

    def _register_hooks(self):
        layer = self._get_layer()
        layer.register_forward_hook(self._forward_hook)
        # Use full backward hook properly
        layer.register_full_backward_hook(self._backward_hook)

    def visualize(self, image_tensor):
        image_tensor = image_tensor.unsqueeze(0).requires_grad_(True)

        output = self.model(image_tensor)
        class_score = output[0, output.argmax()]
        self.model.zero_grad()
        class_score.backward()

        act = self.activations[0, self.target_filter].cpu().numpy()
        grad = self.gradients[0, self.target_filter].cpu().numpy()

        # "Deconv" как прямое перемножение ReLU(активация) и ReLU(градиент)
        act[act < 0] = 0
        grad[grad < 0] = 0
        response = act * grad

        response = (response - response.min()) / (response.max() - response.min() + 1e-8)
        return response


def plot_response(response_map):
    plt.figure(figsize=(5, 5))
    plt.imshow(response_map, cmap='viridis')
    plt.title("DeconvNet Response")
    plt.axis('off')
    plt.show()


In [34]:
# Выбери 1 изображение из теста
image_tensor = next(iter(test_dataloader))[0][0].to(device)

# Визуализатор для слоя layer4.2.conv3, фильтр 10
visualizer = DeconvNetVisualizer(model, target_layer='layer4.2.conv3', target_filter=10)
response = visualizer.visualize(image_tensor)

# Построить результат
plot_response(response)


TypeError: DeconvNetVisualizer._forward_hook() takes 2 positional arguments but 4 were given